# Read data in

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

import os
import os.path
from utils import read_lastfm

In [11]:
songs, users = read_lastfm(zip_name="data/lastfm-dataset-1K.zip")

# Do some data cleaning...

To improve the data quality and solve the previously identified issues of missing ID's and non-cohesive artist and track names, we employed the MusicBrainz Database. This decision was backed by the fact the original dataset relied on this dataset in order to assign each artist and track with an ID, whenever it existed. To tackle this problem, besides the original dataset, three new versions of it are generated, with different levels of data unification. The unmodified dataset will henceforth be referred to as the **Original Dataset**.

The database was downloaded from [the MusicBrainz website](https://musicbrainz.org/doc/MusicBrainz_Database/Download). This database dumps is only made available "in a format that can be loaded into a local instance of PostgreSQL using a local instance of MusicBrainz Server". This notebook reads in the relevant database dump files and parses them into pandas DataFrames before applying that data towards cleaning our original dataset.

## Clean artist names and IDs

Firstly, we unify the artists' information by filling in missing IDs. To achieve this, we start by extracting the Original Dataset's artist's name and applying a simple transformation to it, in order to remove entries where one artist features another, and trying to fill in the missing ID by matching only the main artist with MusicBrainz's Dataset name and sort name (a variant of the artist's official name, often used when sorting artists by name) fields. For entries still not identified, we match the lower-cased versions of the Original Dataset's artist name with both its lower-cased name and sort name counterparts in the MusicBrainz Database. Finally, to address the variation of artist names, we now merge the Original Dataset with the MusicBrainz Database one, but instead of the names, we merge on the artist ID's, and, on match, replace the artist's name with its official version as defined in MusicBrainz.

This creates the **Artist-Merged Dataset**.

#### Convert part of MusicBrainz's database dump into a DataFrame

In [5]:
if not os.path.isfile('data/musicbrainz_artist_df'):

    # create artists_info_df from musicbrainz database dump
    artists_info = []
    with open('mbdump/mbdump/artist') as f:
        lines = (line.rstrip('\r\n') for line in f)
        for line in lines:
            artists_info.append(line.split('\t'))
            
    artist_df = pd.DataFrame(artists_info, columns=['id', 'gid', 'name', 'sort_name', 'begin_date_year', 
                                                'begin_date_month', 'begin_date_day', 'end_date_year', 
                                                'end_date_month', 'end_date_day', 'ended', 'type', 'gender', 
                                                'area', 'begin_area', 'end_area', 'comment', 'edits_pending', 
                                                'last_updated'])
    
    artist_df = artist_df.set_index('id').replace('\\N', np.NaN)
    artist_df.to_pickle('data/musicbrainz_artist_df')

In [13]:
artist_df = pd.read_pickle('data/musicbrainz_artist_df')

#### Fill in as many missing IDs as possible

Data from MusicBrainz

In [14]:
lowercase_artists = artist_df[['gid', 'name', 'sort_name']].copy()
lowercase_artists['lower_name'] = lowercase_artists['name'].str.lower()
lowercase_artists['sort_name'] = lowercase_artists['sort_name'].str.lower()
lowercase_artists

,gid,name,sort_name,lower_name
id,,,,
805192,8972b1c1-6482-4750-b51f-596d2edea8b1,WIK▲N,wik▲n,wik▲n
371203,49add228-eac5-4de8-836c-d75cde7369c3,Pete Moutso,"moutso, pete",pete moutso
273232,c112a400-af49-4665-8bba-741531d962a1,Zachary,zachary,zachary
101060,ca3f3ee1-c4a7-4bac-a16a-0b888a396c6b,The Silhouettes,"silhouettes, the",the silhouettes
145773,7b4a548e-a01a-49b7-82e7-b49efeb9732c,Aric Leavitt,"leavitt, aric",aric leavitt
...,...,...,...,...
2141638,7319bb3c-a5f8-40ae-8be7-b94423938a6e,Evgeny Shagalov,evgeny shagalov,evgeny shagalov
2141639,6dd6cda2-2555-489f-8cfa-32a4f884105d,Kulto Maldito,kulto maldito,kulto maldito
2132558,7fd2ca21-c878-47eb-a528-469b0967691d,RYYZN,ryyzn,ryyzn


Data from Last.fm

In [15]:
merged_user_history_df = songs.copy()
merged_user_history_df['name_for_merge'] = merged_user_history_df['artist-name'].apply(lambda s: s.lower().split(' feat.')[0])
merged_user_history_df.head()

,timestamp,musicbrainz-artist-id,artist-name,musicbrainz-track-id,track-name,name_for_merge
id,,,,,,
1,2009-05-04 23:08:57+00:00,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007,deep dish
1,2009-05-04 13:54:10+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15),坂本龍一
1,2009-05-04 13:52:04+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15),坂本龍一
1,2009-05-04 13:42:52+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15),坂本龍一
1,2009-05-04 13:42:11+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15),坂本龍一


Just a small aside on how perfect results are impossible. As was stated above, even for a human annotator, the disambiguation of bands with the same name can be very hard or even impossible, especially since it's common for much information to be missing.

In [45]:
# one example of the difficulty of this task
artist_df[artist_df['name'] == 'Underworld']

,gid,name,sort_name,begin_date_year,begin_date_month,begin_date_day,end_date_year,end_date_month,end_date_day,ended,type,gender,area,begin_area,end_area,comment,edits_pending,last_updated
id,,,,,,,,,,,,,,,,,,
1318790,12920d58-73cc-41bd-8ef5-07bb57969ffc,Underworld,Underworld,NaN,NaN,NaN,NaN,NaN,NaN,2,38,NaN,60s Canadian garage rock band,0,2016-01-17 16:56:15.138744+00,f,5076,NaN
1416795,a4249a81-f834-4990-9e48-1d3e6ef6b243,Underworld,Underworld,NaN,NaN,NaN,NaN,NaN,NaN,1,105,1,Italian DJ/producer Carlo Prevale,0,2016-10-02 06:44:46.293544+00,f,NaN,NaN
923,ba2f4f3b-0293-4bc8-bb94-2f73b5207343,Underworld,Underworld,1979,NaN,NaN,NaN,NaN,NaN,2,221,NaN,UK electronic music group,0,2019-12-30 00:46:49.015653+00,f,38112,NaN


Start merging process to fill in missing IDs

In [16]:
merged_user_history_df = (pd.merge(merged_user_history_df, lowercase_artists.drop_duplicates(['lower_name']), # no perfect merging strategy to handle duplicates/same name entities, unfortunately
                                   how='left', left_on='name_for_merge', right_on='lower_name')
                          .drop(['name', 'lower_name', 'sort_name'], axis=1))

merged_user_history_df['musicbrainz-artist-id'].fillna(merged_user_history_df['gid'], inplace=True)
merged_user_history_df.drop(['gid'], axis=1, inplace=True)

In [17]:
merged_user_history_df = (pd.merge(merged_user_history_df, lowercase_artists.drop_duplicates(['sort_name']), # no perfect merging strategy to handle duplicates/same name entities, unfortunately
                                   how='left', left_on='name_for_merge', right_on='sort_name')
                          .drop(['name', 'lower_name', 'sort_name'], axis=1))

merged_user_history_df['musicbrainz-artist-id'].fillna(merged_user_history_df['gid'], inplace=True)
merged_user_history_df.drop(['gid'], axis=1, inplace=True)

#### Unify artists' names

Lastly, let's get rid of "duplicate" artist names, by using only their official "name" value for all of their entries.

In [18]:
# merge on gid and just keep "name"
merged_user_history_df = (pd.merge(merged_user_history_df, lowercase_artists,
                                   how='left', left_on='musicbrainz-artist-id', right_on='gid')
                          .drop(['gid', 'lower_name', 'sort_name'], axis=1))

merged_user_history_df['name'].fillna(merged_user_history_df['artist-name'], inplace=True)
merged_user_history_df.drop(['artist-name', 'name_for_merge'], axis=1, inplace=True)
merged_user_history_df.rename(columns={'name': 'artist-name'}, inplace=True)

In [19]:
merged_user_history_df.head()

,timestamp,musicbrainz-artist-id,musicbrainz-track-id,track-name,artist-name
0,2009-05-04 23:08:57+00:00,f1b1cf71-bd35-4e99-8624-24a6e15f133a,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007,Deep Dish
1,2009-05-04 13:54:10+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,NaN,Composition 0919 (Live_2009_4_15),坂本龍一
2,2009-05-04 13:52:04+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,NaN,Mc2 (Live_2009_4_15),坂本龍一
3,2009-05-04 13:42:52+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,NaN,Hibari (Live_2009_4_15),坂本龍一
4,2009-05-04 13:42:11+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,NaN,Mc1 (Live_2009_4_15),坂本龍一


In [20]:
merged_user_history_df.to_pickle('data/artist_merged_user_history_df')

---

## Clean track names and IDs

A very similar process was done to clean the tracks' information, filling in missing ID's by an analogous process. Instead of removing featured artist, track names featuring "live" (followed usually by a specific date) were modified to contain only the track name, and a no-parenthesis variant was also created. Merges were then performed based on these fields and their MusicBrainz Database track name counterpart. For tracks still unidentified, the lower-cased versions of these fields were used for the matching. This results in the **Doubly-Merged Dataset**.

Finally, to unify track ID's, as with artist ID's, a merge was performed based on matching track ID's between the Doubly-Merged Dataset and the MusicBrainz Database, and, on match, the track names were replaced by their official names. This generates in the fourth and final dataset, the **Renamed Doubly-Merged Dataset**.

#### Convert part of MusicBrainz's database dump into a DataFrame

In [ ]:
if not os.path.isfile('data/musicbrainz_track_df'):

    # create tracks_df from musicbrainz database dump
    tracks_info = []
    with open('mbdump/mbdump/track') as f:
        lines = (line.rstrip('\r\n') for line in f)
        for line in lines:
            tracks_info.append(line.split('\t'))
            
    track_df = pd.DataFrame(tracks_info, columns=['id', 'gid', 'recording', 'medium', 'position', 
                                                'number', 'name', 'artist_credit', 'length', 
                                                'edits_pending', 'last_updated', 'comment'])
    track_df = track_df.set_index('id').replace('\\N', np.NaN)
    track_df.to_pickle('data/musicbrainz_track_df')

In [35]:
track_df = pd.read_pickle('data/musicbrainz_track_df')

In [36]:
track_df.head()

,gid,recording,medium,position,number,name,artist_credit,length,edits_pending,last_updated,comment
id,,,,,,,,,,,
34228823,9b02977e-a03b-4a6b-a9a9-06e722bdcd7a,428644,3254461,1,A1,The Ghost of Tom Joad,813,263000,0,2021-02-21 12:19:56.629604+00,f
81,43da7544-6283-3159-84f9-537fe823a1a7,11,600623,5,5,Five Man Army,4,364306,0,2011-05-16 16:08:20.288158+00,f
99,fa124f9a-d8ea-36a3-bed3-c817fdbe13e2,11,600626,5,5,Five Man Army,4,364306,0,2011-05-16 16:08:20.288158+00,f
108,e56c6d3c-09cf-33a0-81c5-ceade77c35dc,11,600627,5,5,Five Man Army,4,364306,0,2011-05-16 16:08:20.288158+00,f
126,b107859d-d32f-3038-87cc-99353b99dace,11,825915,5,5,Five Man Army,4,364306,0,2011-05-16 16:08:20.288158+00,f


#### Fill in as many missing IDs as possible

In [4]:
artist_df = pd.read_pickle('data/musicbrainz_artist_df')
merged_user_history_df = pd.read_pickle('data/artist_merged_user_history_df')

Data from MusicBrainz

In [22]:
lowercase_tracks = track_df[['gid', 'name']].copy()
lowercase_tracks['lower_name'] = lowercase_tracks['name'].str.lower()

Data from Last.fm

In [23]:
double_merged_user_history_df = merged_user_history_df.copy()
double_merged_user_history_df['lower_track'] = double_merged_user_history_df['track-name'].apply(lambda s: str(s).lower().split('(live')[0].strip())
double_merged_user_history_df['no_parens_track'] = double_merged_user_history_df['track-name'].apply(lambda s: str(s).split('(')[0].strip())
double_merged_user_history_df['no_parens_lower_track'] = double_merged_user_history_df['track-name'].apply(lambda s: str(s).lower().split('(')[0].strip())
double_merged_user_history_df.head()

,timestamp,musicbrainz-artist-id,musicbrainz-track-id,track-name,artist-name,lower_track,no_parens_track,no_parens_lower_track
0,2009-05-04 23:08:57+00:00,f1b1cf71-bd35-4e99-8624-24a6e15f133a,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007,Deep Dish,fuck me im famous (pacha ibiza)-09-28-2007,Fuck Me Im Famous,fuck me im famous
1,2009-05-04 13:54:10+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,NaN,Composition 0919 (Live_2009_4_15),坂本龍一,composition 0919,Composition 0919,composition 0919
2,2009-05-04 13:52:04+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,NaN,Mc2 (Live_2009_4_15),坂本龍一,mc2,Mc2,mc2
3,2009-05-04 13:42:52+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,NaN,Hibari (Live_2009_4_15),坂本龍一,hibari,Hibari,hibari
4,2009-05-04 13:42:11+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,NaN,Mc1 (Live_2009_4_15),坂本龍一,mc1,Mc1,mc1


Once again, perfect results are impossible. Even for a human annotator, the disambiguation of songs with the same name can be very hard or even impossible, especially since it's common for much information to be missing. Furthermore, for the specific situation of tracks, because of different "mediums" (about which [MusicBrainz's database information and schema page](https://musicbrainz.org/doc/MusicBrainz_Database/Schema) had no useful information) it's particularly impossible to distinguish between that different entries with the same artist and track names.

In [76]:
# one example of the difficulty of this task
track_df[track_df['name'] == 'Five Man Army'].head()

,gid,recording,medium,position,number,name,artist_credit,length,edits_pending,last_updated,comment
id,,,,,,,,,,,
81,43da7544-6283-3159-84f9-537fe823a1a7,11,600623,5,5,Five Man Army,4,364306,0,2011-05-16 16:08:20.288158+00,f
99,fa124f9a-d8ea-36a3-bed3-c817fdbe13e2,11,600626,5,5,Five Man Army,4,364306,0,2011-05-16 16:08:20.288158+00,f
108,e56c6d3c-09cf-33a0-81c5-ceade77c35dc,11,600627,5,5,Five Man Army,4,364306,0,2011-05-16 16:08:20.288158+00,f
126,b107859d-d32f-3038-87cc-99353b99dace,11,825915,5,5,Five Man Army,4,364306,0,2011-05-16 16:08:20.288158+00,f
45,8dc6d377-8f15-32d6-8545-1c13f33bbd40,11,1311006,5,5,Five Man Army,4,364306,0,2011-05-16 16:08:20.288158+00,f


Start the merging process to fill in missing IDs

In [24]:
double_merged_user_history_df = (pd.merge(double_merged_user_history_df, lowercase_tracks.drop_duplicates(['name']), # no perfect merging strategy to handle duplicates/same name entities, unfortunately
                                          how='left', left_on='track-name', right_on='name')
                                 .drop(['name', 'lower_name'], axis=1))

double_merged_user_history_df['musicbrainz-track-id'].fillna(double_merged_user_history_df['gid'], inplace=True)
double_merged_user_history_df.drop(['gid'], axis=1, inplace=True)

In [25]:
double_merged_user_history_df = (pd.merge(double_merged_user_history_df, lowercase_tracks.drop_duplicates(['name']), # no perfect merging strategy to handle duplicates/same name entities, unfortunately
                                          how='left', left_on='no_parens_track', right_on='name')
                                 .drop(['name', 'lower_name', 'no_parens_track'], axis=1))

double_merged_user_history_df['musicbrainz-track-id'].fillna(double_merged_user_history_df['gid'], inplace=True)
double_merged_user_history_df.drop(['gid'], axis=1, inplace=True)

In [26]:
double_merged_user_history_df = (pd.merge(double_merged_user_history_df, lowercase_tracks.drop_duplicates(['lower_name']), # no perfect merging strategy to handle duplicates/same name entities, unfortunately
                                          how='left', left_on='lower_track', right_on='lower_name')
                                 .drop(['name', 'lower_name', 'lower_track'], axis=1))

double_merged_user_history_df['musicbrainz-track-id'].fillna(double_merged_user_history_df['gid'], inplace=True)
double_merged_user_history_df.drop(['gid'], axis=1, inplace=True)

In [27]:
double_merged_user_history_df = (pd.merge(double_merged_user_history_df, lowercase_tracks.drop_duplicates(['lower_name']), # no perfect merging strategy to handle duplicates/same name entities, unfortunately
                                          how='left', left_on='no_parens_lower_track', right_on='lower_name')
                                 .drop(['name', 'lower_name', 'no_parens_lower_track'], axis=1))

double_merged_user_history_df['musicbrainz-track-id'].fillna(double_merged_user_history_df['gid'], inplace=True)
double_merged_user_history_df.drop(['gid'], axis=1, inplace=True)

In [28]:
double_merged_user_history_df.head()

,timestamp,musicbrainz-artist-id,musicbrainz-track-id,track-name,artist-name
0,2009-05-04 23:08:57+00:00,f1b1cf71-bd35-4e99-8624-24a6e15f133a,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007,Deep Dish
1,2009-05-04 13:54:10+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,11e397fa-0c7d-3ede-bea2-0b55cd813091,Composition 0919 (Live_2009_4_15),坂本龍一
2,2009-05-04 13:52:04+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,3247d6dd-274d-4275-b2e0-7fae5a994d25,Mc2 (Live_2009_4_15),坂本龍一
3,2009-05-04 13:42:52+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,3ed802d8-71e4-30f5-94cf-c653129d42f6,Hibari (Live_2009_4_15),坂本龍一
4,2009-05-04 13:42:11+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,91259686-94cf-316d-aace-2a9aa87421df,Mc1 (Live_2009_4_15),坂本龍一


Measure how much of a difference this merging did.

In [31]:
merged_user_history_df['musicbrainz-track-id'].isna().sum()

2162726

In [32]:
double_merged_user_history_df['musicbrainz-track-id'].isna().sum()

802874

Save the Doubly-Merged Dataset.

In [33]:
double_merged_user_history_df.to_pickle('data/artist_tracks_merged_user_history_df')

Finally, get rid of "duplicate" track names (and instead using only their official "name" value for all of their entries, by "merging" using the gid), and create the fourth and final dataset, Renamed Doubly-Merged Dataset.

In [39]:
# # merge on gid and just keep track "name"
renamed_double_merged_user_history_df = (pd.merge(double_merged_user_history_df, lowercase_tracks, # no need for .drop_duplicates(['gid'])
                                              how='left', left_on='musicbrainz-track-id', 
                                              right_on='gid')
                                         .drop(['lower_name', 'gid'], axis=1))

renamed_double_merged_user_history_df['no_parens_track'] = renamed_double_merged_user_history_df['track-name'].apply(lambda s: str(s).split('(')[0].strip())
renamed_double_merged_user_history_df['name'].fillna(renamed_double_merged_user_history_df['no_parens_track'], inplace=True)
renamed_double_merged_user_history_df.drop(['track-name', 'no_parens_track'], axis=1, inplace=True)
renamed_double_merged_user_history_df.rename(columns={'name': 'track-name'}, inplace=True)
renamed_double_merged_user_history_df['track_name'] = renamed_double_merged_user_history_df['track_name'].replace('NaN', np.NaN)

In [40]:
renamed_double_merged_user_history_df.to_pickle('renamed_artist_track_double_merged_user_history_df')